# Task: To implement Bagging technique using 100 decision trees

In [3]:
# importing necessary libraries
import pandas as pd
import numpy as np

In [4]:
# importing data
data = pd.read_csv('churn_prediction_simple.csv')
data.head()

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,1,3135,66,0,0.0,0,187.0,2,755,224.0,...,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,6,2531,42,0,2.0,0,1494.0,3,388,58.0,...,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1
2,7,263,42,1,0.0,0,1096.0,2,1666,60.0,...,16059.34,15211.29,13798.82,0.36,0.36,857.50,286.07,15719.44,15349.75,0
3,8,5922,72,0,0.0,1,1020.0,1,1,98.0,...,7714.19,7859.74,11232.37,0.64,0.64,1299.64,439.26,7076.06,7755.98,0
4,9,1145,46,0,0.0,0,623.0,2,317,172.0,...,8519.53,6511.82,16314.17,0.27,0.27,443.13,5688.44,8563.84,5317.04,0


In [5]:
predictors = data.drop(columns=['churn','customer_id'])
target = data['churn']

## Steps to implement bagging technique with "n" number of trees
1. Determine the (n_trees) number of trees
2. Make (n_trees) number of bootstrap samples
3. For each bootstrap samples build a decision tree model and generate predictions
4. For every observation in test set, calculate the model of predictions made by n_trees
5. Calculate F1 score for the final predictions

In [6]:
#splitting data
# Split data into training and testing sets (75% for training, 25% for testing)
train_size = 0.75
train = data.sample(frac=train_size, random_state=42)  # Set random state for reproducibility
test = data[~data.index.isin(train.index)] 
#check
train.shape, test.shape

((16550, 21), (5517, 21))

In [7]:
# step1 : Setting n_trees_trees
n_trees = 100

In [18]:
# step2 : Making n_trees bootstrap samples (hint: search pd.sample() function)

def BootStrap(data, n_samples, fraction = 1):
    '''
    function to generate boot strap samples.
    data : data to generate bootstrap samples from
    n_samples : number of samples to create
    fraction : what fraction of data should be each sample (default=1)

    Return: a numpy matrix of N,M elements. where N=samples and M=observation in each sample
    '''
    
     # Calculate the number of samples per row
    samples_per_row = int(len(data) * fraction)
    if samples_per_row < 1:
        samples_per_row = 1  # Handle edge case where fraction is very small

    # Generate bootstrap samples using pandas.sample with replacement
    bootstrap_samples = np.array([
        data.sample(samples_per_row, replace=True).values for _ in range(n_samples)
    ])

    return bootstrap_samples

In [33]:
# Testing function (do not change)
print('observations, columns, samples')
for shape in [BootStrap(train, n_trees, fraction=f).shape for f in [1, 0.75, 0.5]]:
    print('({}, {}, {})'.format(shape[1], shape[2], shape[0]))

observations, columns, samples
(16550, 21, 100)
(12412, 21, 100)
(8275, 21, 100)


### Expected Outcome

<img src="images/image1.png">

In [34]:
# Step3.1 : create a function called generate predictions which will train a decision tree model over one bootstrap sample and return its predictions on test set
from sklearn.tree import DecisionTreeClassifier

def generate_predictions(train_x, train_y, test_x):
    '''
    train_x: independent variables of sample
    train_y: target variable of sample
    test_x: independent variables of test data

    Return: prediction for the test_x
    '''
    # Initialize a DecisionTreeClassifier
    model = DecisionTreeClassifier()

    # Train the model using the training data
    model.fit(train_x, train_y)

    # Generate predictions on the test data
    predictions = model.predict(test_x)

    return predictions

In [35]:
## testing function
print('Expected length of predictions:', test.shape[0])
print("generated predictions shape: ",generate_predictions(train.drop(columns=['churn']), train['churn'], test.drop(columns=['churn'])).size)
# output numbers should be same

Expected length of predictions: 5517
generated predictions shape:  5517


In [47]:
#Step3.2 : Create a function called Bagging which uses the generate_prediction and bootstrap functions to generate predictions for all the bootstrap samples and calculate mode of predictions for eachhh sample.

from scipy.stats import mode
def Bagging(n_trees, train_data, test_data, sample_fraction):
    '''
    N-trees: number of trees
    Return: final predictions of overall bagging technique
    '''
     
    train_x = train_data.iloc[:, :-1]
    train_y = train_data.iloc[:, -1]
    test_x = test_data.iloc[:, :-1]
    test_y = test_data.iloc[:, -1]
    
    predictions = []

    for _ in range(n_trees):
         bootstrap_sample = train_data.sample(frac=sample_fraction, replace=True)
    bootstrap_x = bootstrap_sample.iloc[:, :-1]
    bootstrap_y = bootstrap_sample.iloc[:, -1]
    pred = generate_predictions(bootstrap_x, bootstrap_y, test_x)
    predictions.append(pred)
    final_predictions = mode(np.array(predictions), axis=0)[0]

    return final_predictions

In [48]:
(Bagging(n_trees, train, test, 1))

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)